So basically a merge statement gives you the ability to insert any new records being received.
Update any existing records for which new data has been received.

In [ ]:
#the tgt is where we are going to write the data to and upd is the updates that have come in

%sql
MERGE INTO f1_demo.drivers_merge tgt
USING drivers-day1 upd
ON tgt.driverId = upd.driverId
WHEN MATCHED THEN
    UPDATE SET tgt.dob = upd.dob,
               tgt.forename = upd.forename,
               tgt.surname = upd.surname,
               tgt.updateDate = current_timestamp
    WHEN NOT MATCHED
        THEN INSERT (driverId, dob, forename, surname, createdDate) VALUES (driverId, dob, forename, surname, current_timestamp)

if there is a match then set(update) the value. or else insert a new record

In [ ]:
from pyspark.sql.functions import current_timestamp
from delta.tables import DeltaTable

deltaTable = DeltaTable.forPath(spark, "/mnt/formula1dl/demo/drivers_merge")

deltaTable.alias("tgt").merge(
    drivers_day3_df.alias("upd"),
    "tgt.driverId = upd.driverId" \
   .whenMatchedUpdate(set = {"dob" : "upd.dob", "forename" : "upd.forename", "surname": "upd.surname", "updateDate": "current_timestamp()"}) \
   .whenNotMatchedInsert(values ={
       "driverId": "upd.dirverId",
       "dob": "upd.dob",
       "forename": "upd.forename",
       "surname": "upd.surname",
       "createdDate": "current_timestamp()"
   })
).execute()